In [1]:
import os
import json
import psycopg2

def get_pg_conn_params(profile: str = "default") -> dict:
    """Read PostgreSQL connection parameters from ~/.pg/connection.json."""
    conn_path = os.path.expanduser("~/.pg/connection.json")
    with open(conn_path) as f:
        data = json.load(f)
    if profile not in data:
        raise ValueError(f"No connection profile '{profile}' in {conn_path}")
    return data[profile]

def run_pg_query(query: str, profile: str = "default") -> list:
    """Run a SQL query on PostgreSQL and return results as list of dicts."""
    params = get_pg_conn_params(profile)
    conn = psycopg2.connect(
        host=params["host"],
        port=params.get("port", 5432),
        user=params["user"],
        password=params["password"],
        dbname=params["dbname"]
    )
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            columns = [desc[0] for desc in cur.description]
            rows = cur.fetchall()
            return [dict(zip(columns, row)) for row in rows]
    finally:
        conn.close()

In [3]:
run_pg_query("SELECT * FROM emp", profile="default")

[{'slno': 10, 'description': 'hello'},
 {'slno': 20, 'description': 'world'},
 {'slno': 20, 'description': 'world'}]

In [4]:
run_pg_query("SELECT * FROM items", profile="default")

[{'id': 1, 'embedding': '[1,2,3]'}, {'id': 2, 'embedding': '[4,5,6]'}]

In [5]:
run_pg_query("SELECT * FROM playing_with_neon LIMIT 10", profile="default")

[{'id': 1, 'name': 'c4ca4238a0', 'value': 0.72201073},
 {'id': 2, 'name': 'c81e728d9d', 'value': 0.5568709},
 {'id': 3, 'name': 'eccbc87e4b', 'value': 0.45033464},
 {'id': 4, 'name': 'a87ff679a2', 'value': 0.99806464},
 {'id': 5, 'name': 'e4da3b7fbb', 'value': 0.29121268},
 {'id': 6, 'name': '1679091c5a', 'value': 0.39245477},
 {'id': 7, 'name': '8f14e45fce', 'value': 0.12164197},
 {'id': 8, 'name': 'c9f0f895fb', 'value': 0.055121623},
 {'id': 9, 'name': '45c48cce2e', 'value': 0.71753603},
 {'id': 10, 'name': 'd3d9446802', 'value': 0.7876193}]

In [8]:
def run_pg_insert(insert_sql: str, profile: str = "default") -> int:
    """Run an INSERT statement on PostgreSQL. Returns number of rows inserted."""
    params = get_pg_conn_params(profile)
    conn = psycopg2.connect(
        host=params["host"],
        port=params.get("port", 5432),
        user=params["user"],
        password=params["password"],
        dbname=params["dbname"]
    )
    try:
        with conn.cursor() as cur:
            cur.execute(insert_sql)
            conn.commit()
            return cur.rowcount
    finally:
        conn.close()

In [9]:
# Example: Insert a new employee into the 'emp' table
insert_sql = "INSERT INTO emp (slno, description) VALUES (100, 'awesome') "
rows_inserted = run_pg_insert(insert_sql, profile="default")
print(f"Rows inserted: {rows_inserted}")

Rows inserted: 1
